In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import print_breakdown, search

# TODO: handle non-US cars
df_nhtsa = await search(
  min_model_year=2019,  # TODO: increase range later
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36766 VINs (filter_comment=None, include_openpilot=True, skipped=162, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/32344 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              997
Bronco Sport       1500
Continental          23
Corsair             986
E-Transit           157
Ecosport            322
Edge               2450
Escape             3133
Expedition          725
Expedition MAX      629
Explorer           2971
F-150              6541
F-150 Lightning     385
F-250              1500
F-350              1034
F-450               196
F-550                 1
Fiesta              147
Flex                137
Fusion              908
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             871
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus               40
Transit            1009
Transit Connect     340
dtype: int64


In [2]:
df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df.head()

,VIN,CarName
0,1FTFX1E56PKD13147,FORD F-150 2023
1,1FTEW1EPXKFA72520,FORD F-150 2019
2,1FBAX2X89PKA13324,FORD Transit 2023
4,1FTEW1EP5MKD05620,FORD F-150 2021
5,1FTEW1CP4PKD70482,FORD F-150 2023


In [3]:
import pandas as pd

from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu

ecu_map = {
  'ABS': FordEcu.AntiLockBrakeSystem,
  'APIM': FordEcu.AccessoryProtocolInterfaceModule,
  'IPMA': FordEcu.ImageProcessingModuleA,
  'PSCM': FordEcu.PowerSteeringControlModule,
}

def get_ecu_platform_code(abd: AsBuiltData, ecu: FordEcu) -> str | None:
  fw = abd.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if fw is None:
    return None
  prefix, core, _ = fw.split('-')
  return f'{core}-{prefix}'

def get_platform_codes(row) -> pd.Series:
  abd = AsBuiltData.from_vin(row['VIN'])
  values: dict[str, str | None] = {}
  for ecu_name, ecu in ecu_map.items():
    values[ecu_name] = get_ecu_platform_code(abd, ecu)
  return pd.Series(values)

df_fw = df.join(df.apply(get_platform_codes, axis=1))
df_fw.head()

,VIN,CarName,ABS,APIM,IPMA,PSCM
0,1FTFX1E56PKD13147,FORD F-150 2023,2D053-PL34,14G676-PU5T,14H102-PJ6T,14D003-ML3V
1,1FTEW1EPXKFA72520,FORD F-150 2019,2D053-KL34,14G374-1U5T,14F397-KL3T,14D003-KL3V
2,1FBAX2X89PKA13324,FORD Transit 2023,2D053-NK41,14G676-NU5T,14F397-NK3T,14D003-KK21
4,1FTEW1EP5MKD05620,FORD F-150 2021,2D053-ML34,14G676-MU5T,14H102-ML3T,14D003-MB3C
5,1FTEW1CP4PKD70482,FORD F-150 2023,2D053-PL34,14G676-PU5T,14H102-PJ6T,14D003-ML3V


In [4]:
from typing import TypeVar

T = TypeVar('T')

class UnionFind:
  def __init__(self, n: int):
    self.parent = list(range(n))

  def find(self, x: int) -> int:
    if self.parent[x] != x:
      self.parent[x] = self.find(self.parent[x])
    return self.parent[x]

  def union(self, x: int, y: int) -> None:
    rootX = self.find(x)
    rootY = self.find(y)
    if rootX != rootY:
      self.parent[rootY] = rootX

def merge_sets(sets: list[set[T]]) -> list[set[T]]:
  element_to_index = {}
  uf = UnionFind(len(sets))

  for i, s in enumerate(sets):
    for element in s:
      if element in element_to_index:
        uf.union(i, element_to_index[element])
      element_to_index[element] = i

  merged = {}
  for i, s in enumerate(sets):
    root = uf.find(i)
    if root in merged:
      merged[root] |= s
    else:
      merged[root] = s.copy()

  return list(merged.values())

In [5]:
def group_by(df: pd.DataFrame, by: list[str]) -> None:
  car_groups = list()
  for group in df.groupby(by, dropna=False):
    cars = set(group[1]['CarName'].unique())
    car_groups.append(cars)

  return merge_sets(car_groups)

In [12]:
from notebooks.ford.platforms import find_openpilot_platform

for ecus in (
  # ('ABS', 'PSCM'), - combines escape and bronco sport
  # ('ABS', 'APIM'), - combines escape and bronco sport
  # ('IPMA', 'PSCM'),
  # ('IPMA', 'APIM'),
  # ('PSCM', 'IPMA', 'APIM'), - combines escape and bronco sport
  # ('ABS', 'PSCM', 'APIM'),  - combines escape and bronco sport
  # ('ABS', 'IPMA', 'APIM'),
  ('ABS', 'IPMA'),
  ('ABS', 'IPMA', 'PSCM'),  # splits out: 2019 Mustang (from 2020-23), 2024 Bronco (from 2021-23)
  ('ABS', 'IPMA', 'PSCM', 'APIM'),  # splits out: 2019 EcoSport (from 2020-22), 2024 Escape/Corsair (from 2023)
):
  car_groups = group_by(df_fw, list(ecus))
  print(', '.join(ecus))
  print(f'Found {len(car_groups)} distinct car groups')
  for group in car_groups:
    platforms = {find_openpilot_platform(car) for car in group}
    print(f'{len(platforms)} platforms: {", ".join(map(str, platforms)):<33}; {", ".join(group)}')
  print()

ABS, IPMA
Found 50 distinct car groups
1 platforms: None                             ; FORD Transit 2019
1 platforms: None                             ; FORD Escape 2019
1 platforms: None                             ; FORD Ranger 2019, FORD Ranger 2022, FORD Ranger 2020, FORD Ranger 2021, FORD Ranger 2023
1 platforms: None                             ; FORD Fiesta 2019
1 platforms: None                             ; FORD Flex 2019, LINCOLN MKT 2019, FORD Taurus 2019
1 platforms: None                             ; LINCOLN Continental 2020, LINCOLN Continental 2019
1 platforms: None                             ; FORD Ecosport 2020, FORD Ecosport 2022, FORD Ecosport 2019, FORD Ecosport 2021
1 platforms: None                             ; FORD Explorer 2019
1 platforms: None                             ; FORD GT 2020
1 platforms: None                             ; FORD Expedition 2019, FORD Expedition MAX 2019, LINCOLN Navigator L 2019, LINCOLN  2019, LINCOLN Navigator 2019
1 platforms: FO